In [ ]:
#r "nuget: Microsoft.Extensions.Configuration, 8.0.0"
#r "nuget: Microsoft.Extensions.Configuration.FileExtensions, 8.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Json, 8.0.0"
#r "nuget: AWSSDK.BedrockRuntime, 3.7.302.7"
#r "nuget: Rockhead.Extensions, 0.1.2-preview"

# Use Access Keys and call Foundational model from Amazon Bedrock

In [ ]:
using System.IO;
using Microsoft.Extensions.Configuration;
using System.Text.Json;
using System.Text.Json.Nodes;
using System.Text.Json.Serialization;
using Amazon;
using Amazon.Util;
using Amazon.BedrockRuntime;
using Amazon.BedrockRuntime.Model;
using Rockhead.Extensions; // TODO: consider contributing


var builder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile(Path.Combine(
        Environment.GetFolderPath(Environment.SpecialFolder.ApplicationData),
        "Microsoft",
        "UserSecrets",
        "2b6c897c-02c0-4c8e-91a8-cb61113a6acc", // comes from the project file QuokkaVec.Console.csproj
        "secrets.json")
    );

IConfigurationRoot configuration = builder.Build();

string awsAccessKeyId = configuration["AWSAccessKeyId"] ?? string.Empty;
string awsSecretAccessKey = configuration["AWSSecretAccessKey"] ?? string.Empty;

if (string.IsNullOrWhiteSpace(awsAccessKeyId) || string.IsNullOrWhiteSpace(awsSecretAccessKey))
{
	Console.WriteLine("AWSAccessKeyId or AWSSecretAccessKey is not set, use 'dotnet user-secrets init' and 'dotnet user-secrets set <key> <value>' to set the key, exiting...");
	return;
}

string modelId = "meta.llama3-70b-instruct-v1:0";
AmazonBedrockRuntimeClient awsClient = new(awsAccessKeyId, awsSecretAccessKey, RegionEndpoint.USEast1);
string payload = new JsonObject()
{
	{ "prompt", "Test prompt" },
	{ "max_gen_len", 512 },
	{ "temperature", 0.5 }
}.ToJsonString();

try
{
	var response = await awsClient.InvokeModelAsync(new InvokeModelRequest
	{
		ModelId = modelId,
		Body = AWSSDKUtils.GenerateMemoryStreamFromString(payload),
		ContentType = "application/json",
		Accept = "application/json"
	});
	Console.WriteLine(response);
	Console.WriteLine(response?.HttpStatusCode);
	Console.WriteLine(response?.Body?.ToString());

	// use system.text.json to parse the response into testResponse object
	var jsonString = await new StreamReader(response.Body).ReadToEndAsync();
	Console.WriteLine(jsonString);

	var testResponse = JsonSerializer.Deserialize<Llama2Response>(jsonString) ?? new Llama2Response();
	Console.WriteLine(testResponse);
	Console.WriteLine(testResponse.Generation);
	Console.WriteLine(testResponse.PromptTokenCount);
	Console.WriteLine(testResponse.GenerationTokenCount);
	Console.WriteLine(testResponse.StopReason);
}
catch (Exception ex)
{
	Console.WriteLine($"Error invoking model: {ex.Message}");
}